# Advanced Usage I

## Complex Types

We have already seen how to define classes / models using simple types, such as `int`, `float`, `str`, `bool`, etc. However in practice data structures are often more complex. They include for examples dictionaries, list of lists, or mutiple allowed types. Of course Pydantic also supports these more complex types, such as lists, dictionaries, enums, and unions. In the following we will see an overview of those types and how to use them:


### Typed Lists and Dictionaries

Lists and dictionaries are very common data structures in Python. Pydantic supports typed lists and dictionaries, which means that we can also define the type of the elements in the list or the type of the values in the dictionary.
Typed lists and dictionaries are defined using the `list` and `dict` generic types. For example, we can define a model with a list of floats as follows:

In [ ]:
from pydantic import BaseModel


class LineV1(BaseModel):
    """A Line object represented by two lists of coordinates"""

    x: list[float]
    y: list[float]

    def length(self):
        """Line length computed by summing over the distance between points"""
        length = 0

        for idx in range(len(self.x) - 1):
            length += (
                (self.x[idx] - self.x[idx + 1]) ** 2
                + (self.y[idx] - self.y[idx + 1]) ** 2
            ) ** 0.5

        return length

Instanciation of the model works as expected:

In [ ]:
line_v1 = LineV1(x=[0, 1, 3], y=[0, 1, 2])
display(line_v1)

In [ ]:
print(line_v1.length())

The behavior is exactly the same as for simple types. So values are converted to the specified type if possible:

In [ ]:
line_v1 = LineV1(x=[0, 1, "3"], y=[0, True, 2])
display(line_v1)

If the type cannot be converted a `ValidationError` is raised:

In [ ]:
line_v1 = LineV1(x=[0, 1, "10^-2"], y=[0, 1, 2])
display(line_v1)

Note that pydantic indicates the index of the invalid value in the error message using `x -> 2`.

Typed dictionaries are defined in a similar way using the `dict` generic type:

In [ ]:
from pydantic import BaseModel


class LineV2(BaseModel):
    """A Line object represented by two lists of coordinates"""

    coordinates: dict[str, list[float]]

    def length(self):
        """Line length computed by summing over the distance between points"""
        length = 0
        x = self.coordinates["x"]
        y = self.coordinates["y"]

        for idx in range(len(x) - 1):
            length += ((x[idx] - x[idx + 1]) ** 2 + (y[idx] - y[idx + 1]) ** 2) ** 0.5

        return length

In [ ]:
data = {
    "x": [0, 1, 3],
    "y": [0, 1, 2],
}

### Enums and Union Types

In many cases it is useful to provide users with a selection of valid values, such as a choice of strings. The data structure to handle this is called `Enum`. Enums are defined using the `Enum` generic type in Python. For example, we can define a selection for the line color:


In [ ]:
from pydantic import BaseModel
from enum import Enum


class LineColor(str, Enum):
    """Line color enum"""

    red = "red"
    green = "green"
    blue = "blue"

And add this to the definiton of the line class:

In [ ]:
class ColoredLine(BaseModel):
    """A Line object that can be used to represent a line."""

    x: list[float]
    y: list[float]
    color: LineColor = LineColor.red

    def length(self):
        """Length of the line"""
        length = 0

        for idx in range(len(self.x) - 1):
            length += (
                (self.x[idx] - self.x[idx + 1]) ** 2
                + (self.y[idx] - self.y[idx + 1]) ** 2
            ) ** 0.5

        return length

On initialisation we can now pass a color:

In [ ]:
colored_line = ColoredLine(x=[0, 1, 2], y=[0, 1, 2], color="red")
display(colored_line)

Now let's try an invalid color:

In [ ]:
colored_line = ColoredLine(x=[0, 1, 2], y=[0, 1, 2], color="purple")

Pydantic now gives a useful error message, saying that the value is not a valid enumeration member; and it also lists the valid choices.

**Note:** that there is https://github.com/pydantic/pydantic-extra-types, which provides support for validation of CSS colors. `from pydantic.color import Color`, so there is actually no reason to implement this ourselves.

In [ ]:
from typing import Union

from pydantic import BaseModel
from enum import Enum


class LineColor(str, Enum):
    """Line color enum"""

    red = "red"
    green = "green"
    blue = "blue"


class LineV2(BaseModel):
    """A Line object that can be used to represent a line."""

    x: list[float]
    y: list[float]
    color: Union[LineColor, None] = LineColor.red


### Datetime Types

Another very useful advanced type is the `datetime`, which lets users handle dates and times. Pydantic natively supports parsing and validation of datetime types. This relies on the Python standard library (see https://docs.python.org/3/library/datetime.html). Let's take a look at an example:



In [ ]:
from datetime import datetime


class LonLatTimeVector(BaseModel):
    """Represents a position on earth with time."""

    lon: float = 0.0
    lat: float = 0.0
    time: datetime = None


position = LonLatTimeVector(lon=1.0, lat=2.0, time=datetime.now())
display(position)

In [ ]:
type(position.time)

In [ ]:
position.time.isoformat()

`datetime.now()` already creates a `datetime` object, but pydantic also supports other valid formats, such as:

In [ ]:
# Time in ISO format
position = LonLatTimeVector(time="2021-01-01T00:00:00")
display(position)

In [ ]:
# Ints or floats interpred as unix time, i.e. seconds since 1970-01-01T00:00:00
position = LonLatTimeVector(time=1609459200)
display(position)

Let's try to pass an invalid datetime:

In [ ]:
position = LonLatTimeVector(time="2021-01-01")

### Other Types

There is a variety of other useful types built into pydantic. We have e.g. already mentioned the `Color` type above. Most of these types are mostly relevant for web development but, non-the-less, it is good to know that they exist. For an overview of those types see https://docs.pydantic.dev/latest/usage/types/#pydantic-types and also https://github.com/pydantic/pydantic-extra-types (unreleased).

### Custom Data Types / Hierarchical Structures

One of the most powerful features of Pydantic is the ability to combine models to create hierarchical structures. This is done simply by using an existing pydantix model as type. For example we easily define a triangle using three points:


In [ ]:
from pydantic import BaseModel


class Point(BaseModel):
    """Representation of a two-dimensional point coordinate."""

    x: float
    y: float

    def distance_to(self, other: "Point") -> float:
        """Computes the distance to another `PointV3`."""
        dx = self.x - other.x
        dy = self.y - other.y
        return (dx**2 + dy**2) ** 0.5


class Triangle(BaseModel):
    """Representation of a triangle"""

    point_a: Point = Point(x=0, y=0)
    point_b: Point = Point(x=1, y=0)
    point_c: Point = Point(x=0, y=1)

    def circumference(self):
        """Circumference of the triangle"""
        return (
            self.point_a.distance_to(self.point_b)
            + self.point_b.distance_to(self.point_c)
            + self.point_c.distance_to(self.point_a)
        )


triangle = Triangle(
    point_a=Point(x=0, y=0),
    point_b=Point(x=0.5, y=0),
    point_c=Point(x=0, y=0.5),
)

display(triangle)

In [ ]:
print(triangle.circumference())

Alternatively in this case we can also pass a dictionary to the model without creating the `Point` instances first:

In [ ]:
triangle = Triangle(
    point_a={"x": 0, "y": 0.5},
    point_b={"x": 0.5, "y": 0},
    point_c={"x": 0, "y": 0},
)

print(triangle.circumference())

Pydantic will automatically convert the dictionaries to `Point` instances by passing the arguments to `Point(**kwargs)`. This is already a preview of the serialization and deserialization of models into dictionaries and hierarchical languages such as JSON, YAML and TOML. Let's quickly verify the type of the point:

In [ ]:
type(triangle.point_a)

Of course this also works for list of `Point` objects. For example we can rewrite the definition of the `Line` class we introduced above as follows:

In [ ]:
from pydantic import BaseModel


class Point(BaseModel):
    """Representation of a two-dimensional point coordinate."""

    x: float
    y: float

    def distance_to(self, other: "Point") -> float:
        """Computes the distance to another `PointV3`."""
        dx = self.x - other.x
        dy = self.y - other.y
        return (dx**2 + dy**2) ** 0.5


class LineV3(BaseModel):
    """Line object represented by a list of points"""

    points: list[Point]

    def length(self):
        """Line length computed by summing over the distance between points"""
        length = 0

        for point, next_point in zip(self.points[:-1], self.points[1:]):
            length += point.distance_to(next_point)

        return length

Which can now be used as follows:

In [ ]:
line_v3 = LineV3(points=[Point(x=0, y=0), Point(x=1, y=1)])
display(line_v3)

In [ ]:
print(line_v3.length())

If you compare to our first implementation at the beginning the `LineV3` is more compact, better readable and more elegant. 

### Private Attributes

By default all attributes of a Pydantic model are public. However, sometimes we want to define private attributes, which are not accessible from outside the model. This can be done using the `ClassVar` and `PrivateAttr` types. The `ClassVar` type is used to define class variables, which are shared by all instances of the class. The `PrivateAttr` type is used to define private attributes, which are not accessible from outside the model. The following example shows how to use these types in more detail:

In [ ]:
class LineV4(BaseModel):
    """Line object represented by a list of points"""

    points = list[Point]

    def length(self):
        """Line length computed by summing over the distance between points"""
        length = 0

        for point, next_point in zip(self.points[:-1], self.points[1:]):
            length += point.distance_to(next_point)

        return length

`Config.underscore_attrs_are_private` is `True`